# Hack [nairaland.com](https://www.nairaland.com/)

Several demos of this project in action are provided below. Just change the cell type to `code` to run the examples.

In [1]:
import hack

In [ ]:
hack.export_user_comments_to_html(username="seun", max_page=10)

In [2]:
import textwrap

user_comments = hack.UserCommentHistory("preccy69")
for page in list(user_comments.scrap_comments_for_range_of_pages(start=0, stop=1)):
    for section, topic_plus_comment in page.items():
        print("\n\n", "*"*40, section, "*"*40)
        print(topic_plus_comment.topic.upper()) # for differentiation only

        parsed_comment = topic_plus_comment.parsed_comment # a namedtuple instance
        print(parsed_comment.focus_user_comment)

        quotes = parsed_comment.quotes_ordered_dict
        for username, comment in quotes.items():
            print(" "*8)
            print(textwrap.indent(username, "    "))
            print(textwrap.indent(comment, "    "))
            
        print("_"*100)
    print("\n\n")



 **************************************** Celebrities ****************************************
LILIAN ESORO AND FRIENDS RECREATE THROWBACK PHOTO
for a sec I tot d lady on d left is lilian
____________________________________________________________________________________________________


 **************************************** Romance ****************************************
UPLOAD YOUR PHONE HOME SCREENSHOT LET'S RATE IT
am fyn thank you
        
    Mavin1
    Fine geh wasup?
____________________________________________________________________________________________________


 **************************************** Romance**4 ****************************************
UPLOAD YOUR PHONE HOME SCREENSHOT LET'S RATE IT
ok then
____________________________________________________________________________________________________


 **************************************** Career ****************************************
NINE THINGS NIGERIAN BANKERS HATE ABOUT THEIR JOBS
what about hav

In [3]:
import textwrap

p = hack.TopicCollector(section='politics')
for page in p.scrap_topics_for_range_of_pages(end=1):
    for topic in list(page):
        print(topic.poster)
        print(textwrap.indent(topic.title, "    "))
        print(textwrap.indent(topic.url, "    "))
        print(textwrap.indent(str(topic.comments), "    "), " comments")
        print()

Seun
    Nairaland Says No To Secessionists
    http://www.nairaland.com/2792995/nairaland-says-no-secessionists
    3214  comments

Tetehjewels
    More Defection Hits Ikono PDP Chapter As Thousands Dump Party For APC
    http://www.nairaland.com/4865195/more-defection-hits-ikono-pdp
    11  comments

YouthsPC0
    157 Soldiers Massacred Same Day Service Chiefs Were At APC Rally - Reno
    http://www.nairaland.com/4865142/157-soldiers-massacred-same-day
    76  comments

SouthWestBlood
    #TakeItBack Movement: Omoyele Sowore Campaigns In Kano (Pictures)
    http://www.nairaland.com/4865013/takeitback-movement-omoyele-sowore-campaigns
    69  comments

TonyeBarcanista
    Kenya Is Ahead of Nigeria In All Aspect (Facts Don't Lie)
    http://www.nairaland.com/3327789/kenya-ahead-nigeria-all-aspect
    79938  comments

anambraeyes
    Jibrin Of Sudan And The Audacity Of Fake News
    http://www.nairaland.com/4850545/jibrin-sudan-audacity-fake-news
    335  comments

OEPHIUS
    Death Of 

Christian590
    Soldiers Massacare: Buhari Sends Defense Minister To Meet Chad's President
    http://www.nairaland.com/4864377/soldiers-massacare-buhari-sends-defense
    173  comments

sarrki
    34 Suspected Members Of IPOB Arrested Over Death Of Police Inspector
    http://www.nairaland.com/4864980/34-suspected-members-ipob-arrested
    13  comments

Woyinpreye123
    Akwa Ibom Government To Share Free Fuel, Food, Air Freshners To Eligible Voters
    http://www.nairaland.com/4864367/akwa-ibom-government-share-free
    49  comments

oviosun
    Manufacturers,artisans,market Women Mobilise To Welcome PMB To Edo State.
    http://www.nairaland.com/4864953/women-mobilise-welcome-pmb-edo
    6  comments

conductor121
    NEWSBREAKING: Boko Haram Kidnaps Another Set Of Girls
    http://www.nairaland.com/4865173/newsbreaking-boko-haram-kidnaps-another
    9  comments

Alakowe2001
    Kwara: How Ibolo Leaders Prevailed On Governor Ahmed To Cede Senate Ticket
    http://www.nairaland.com/4

In [ ]:
# To do
def export_user_comments_to_excel(username=None, max_page=5):
    """Export all of a user's comments data to excel

    Parameters
    -----------
    str
        Username
    int
        Maximum page count for user's comments (Default is ..)
        loop breaks if we exceed actual count
    """
    if not username:
        print("No username provided. Ending")
        return
    else:
        print("Now hacking nairaland. Please wait a few minutes.")
        
    user_data = UserCommentHistory(username).scrap_comments_for_range_of_pages(stop=max_page)
    user_comments = list(user_data)
    work_book = OP.Workbook()
    active_sheet = work_book.active
    active_sheet.title = username
    active_sheet['A1'] = "SECTION"
    active_sheet['B1'] = "TOPIC"
    active_sheet['C1'] = "QUOTED"
    active_sheet['D1'] = "USER COMMENT"

    user = UserCommentHistory(username)
        for page in list(user.scrap_comments_for_range_of_pages(stop=max_page)):
            for section, topic_plus_comment in page.items():
                f.write("<h3>{}</h3>\n".format(section.split('**')[0])) # remove the ** separating section and index
                parsed_comment = topic_plus_comment.parsed_comment
                f.write("<p class='text-success'>{}</p>\n".format(parsed_comment.focus_user_comment))
                quotes = parsed_comment.quotes_ordered_dict
                
                for username, comment in quotes.items():
                    f.write("<h4>{}</h4>\n".format(username))
                    f.write("<p class='text-primary'><em>{}</em></p>\n".format(comment))
                f.write("<div class='dropdown-divider'></div>\n")
            f.write("<hr>\n")
        f.write("</div>\n")
        f.write("</body>")    
    print("Done hacking")

    row_number = 2
    for each_comment in user_comments:

        poster = each_comment[0]
        section = each_comment[1]
        topic = each_comment[2]
        quoted = single_string_from_dictionary(each_comment[3])
        comment = each_comment[4]

        active_sheet.cell(row=row_number, column=1, value=poster)
        active_sheet.cell(row=row_number, column=2, value=section)
        active_sheet.cell(row=row_number, column=3, value=topic)
        active_sheet.cell(row=row_number, column=4, value=quoted)
        active_sheet.cell(row=row_number, column=5, value=comment)

        row_number += 1
    work_book.save(os.path.join(BASE_DIR, "comments_{}_{}_pages.xlsx".format(username.lower(), max_page)))


In [ ]:
# To do
def export_user_comments_to_excel(username=None, max_page=5):
    """Export all of a user's comments data to excel

    Parameters
    -----------
    str
        Username
    int
        Maximum page count for user's comments (Default is ..)
        loop breaks if we exceed actual count
    """
    if not username:
        print("No username provided. Ending")
        return
    else:
        print("Now hacking nairaland. Please wait a few minutes.")
        
    user_data = UserCommentHistory(username).scrap_comments_for_range_of_pages(stop=max_page)
    user_comments = list(user_data)
    work_book = OP.Workbook()
    active_sheet = work_book.active
    active_sheet.title = username
    active_sheet['A1'] = "SECTION"
    active_sheet['B1'] = "TOPIC"
    active_sheet['C1'] = "QUOTED"
    active_sheet['D1'] = "USER COMMENT"

    user = UserCommentHistory(username)
        for page in list(user.scrap_comments_for_range_of_pages(stop=max_page)):
            for section, topic_plus_comment in page.items():
                f.write("<h3>{}</h3>\n".format(section.split('**')[0])) # remove the ** separating section and index
                parsed_comment = topic_plus_comment.parsed_comment
                f.write("<p class='text-success'>{}</p>\n".format(parsed_comment.focus_user_comment))
                quotes = parsed_comment.quotes_ordered_dict
                
                for username, comment in quotes.items():
                    f.write("<h4>{}</h4>\n".format(username))
                    f.write("<p class='text-primary'><em>{}</em></p>\n".format(comment))
                f.write("<div class='dropdown-divider'></div>\n")
            f.write("<hr>\n")
        f.write("</div>\n")
        f.write("</body>")    
    print("Done hacking")

    row_number = 2
    for each_comment in user_comments:

        poster = each_comment[0]
        section = each_comment[1]
        topic = each_comment[2]
        quoted = single_string_from_dictionary(each_comment[3])
        comment = each_comment[4]

        active_sheet.cell(row=row_number, column=1, value=poster)
        active_sheet.cell(row=row_number, column=2, value=section)
        active_sheet.cell(row=row_number, column=3, value=topic)
        active_sheet.cell(row=row_number, column=4, value=quoted)
        active_sheet.cell(row=row_number, column=5, value=comment)

        row_number += 1
    work_book.save(os.path.join(BASE_DIR, "comments_{}_{}_pages.xlsx".format(username.lower(), max_page)))

def title_word_count(section='romance', max_page=10):
    """Group words that occur in titles of a section

    Parameters
    -----------
    str
        Section
    int
        Maximum number of section pages to scrap

    Returns
    --------
    word_count : list
        Dictionary of each word and its count
    list
        A list of all words sorted in descending order of frequency
    """
    word_list = []
    section_object = TopicCollector(section=section).titles_links_metadata(end=max_page)
    section_data = list(section_object)
    split_string = r"[\; \, \n \.+\- \( \) - \/ : \? \[ \] \ — –]"

    exclude = ['to', 'a', 'as', 'the', 'you', 'in', 'is', 'i', 'with', 'of', 'an', 'and', 'my',
               'your', 'for', 'on', 'what', 'her', 'this', 'that', 'these', 'those', 'me']

    for each in section_data:
        title = each[3]
        words_in_title = re.split(split_string, title)
        word_list.extend(words_in_title)

    word_list = [x.lower() for x in word_list]
    word_list = filter(lambda x: x not in exclude, word_list)
    word_count = Counter(list(word_list))
    return word_count, sort_dictionary_by_value(word_count)

def export_topics_to_excel(section='romance', start_page=0, end_page=3):
    """Writes all topics between start and end of a section to excel.
    Same output as titles_links_metadata() but written to a excel file
    """
    work_book = OP.Workbook()
    active_sheet = work_book.active
    active_sheet.title = section
    active_sheet['A1'] = "POSTER"
    active_sheet['B1'] = "MONTH"
    active_sheet['C1'] = "YEAR"
    active_sheet['D1'] = "TITLE"
    active_sheet['E1'] = "WEB URL"

    section_object = TopicCollector(section=section)
    section_titles = section_object.titles_links_metadata(start=start_page, end=end_page)
    section_data = list(section_titles)

    row_number = 2

    for each in section_data:
        title = each[0]
        link = each[1]
        month = each[2]
        year = each[3]
        poster = each[4]

        active_sheet.cell(row=row_number, column=1, value=title)
        active_sheet.cell(row=row_number, column=2, value=link)
        active_sheet.cell(row=row_number, column=3, value=month)
        active_sheet.cell(row=row_number, column=4, value=year)
        active_sheet.cell(row=row_number, column=5, value=poster)
        row_number += 1

    fname = "{}_topics_{}_to_{}.xlsx".format(section, start_page, end_page)
    work_book.save(os.path.join(BASE_DIR, fname))

def export_post_to_docx_format(post_url):
    """Export post to word"""

    post = PostCollector(post_url)
    comments_and_commenters = post.view_comments_commenters()

    document = docx.Document()
    document.add_paragraph(post_url)

    for each in comments_and_commenters:
        commenter = each[0]
        comment_block = each[1]

        quotes = single_string_from_dictionary(comment_block[0])
        user_comments = comment_block[1]

        document.add_paragraph().add_run(commenter).bold = True
        document.add_paragraph().add_run('[{}]'.format(quotes)).italic = True
        document.add_paragraph(user_comments)
        document.add_paragraph('*'*50)

    fname = "{}.docx".format(post.post_title)
    document.save(os.path.join(BASE_DIR, fname))